# View

> Display information from IFC in Panda's DataFrames

In [ ]:
# | default_exp _view

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

In [ ]:
# | export
from fastcore.basics import patch
import ifcopenshell
import numpy as np
import pandas as pd

In [ ]:
import ifctruss._example
import ifctruss.ifctruss

In [ ]:
# | export
class View:
    def __init__(
        self,
        model: ifcopenshell.file,
        *,
        structural_analysis_model: str,
        load_group: str
    ):
        self.ifc_model = model
        self.ifc_structural_analysis_model = self.ifc_model.by_guid(
            structural_analysis_model
        )
        self.load_group = self.ifc_model.by_guid(load_group)

In [ ]:
dfs = {
    "nodes": ifctruss._example.nodes,
    "bars": ifctruss._example.bars,
    "point_loads": ifctruss._example.point_loads,
}

In [ ]:
model = ifctruss.ifctruss.build(**dfs)

In [ ]:
model.by_type("IfcStructuralAnalysisModel")

[#14=IfcStructuralAnalysisModel('0x_8t47bn5NwN19Kz51uq5',$,'Truss model',$,$,.NOTDEFINED.,#9,(#66),$,#13)]

In [ ]:
structural_analysis_model = model.by_type("IfcStructuralAnalysisModel")[
    0
].GlobalId

In [ ]:
model.by_type("IfcStructuralAnalysisModel")[0].LoadedBy

(#66=IfcStructuralLoadGroup('06PfYn$Rr6VQvHsU$2enhS',$,'Load Group 1',$,$,.LOAD_GROUP.,.NOTDEFINED.,.NOTDEFINED.,$,$),)

In [ ]:
load_group = (
    model.by_type("IfcStructuralAnalysisModel")[0].LoadedBy[0].GlobalId
)

In [ ]:
System = View(
    model,
    structural_analysis_model=structural_analysis_model,
    load_group=load_group,
)

In [ ]:
System.load_group

#66=IfcStructuralLoadGroup('06PfYn$Rr6VQvHsU$2enhS',$,'Load Group 1',$,$,.LOAD_GROUP.,.NOTDEFINED.,.NOTDEFINED.,$,$)

In [ ]:
nodes_and_bars = System.ifc_structural_analysis_model.IsGroupedBy[
    0
].RelatedObjects

In [ ]:
nodes_and_bars

(#22=IfcStructuralPointConnection('2pRj5mp2n9meHLovYC9E37',$,'Node 2',$,$,#16,#21,#17,$),
 #28=IfcStructuralPointConnection('0l8conwJL1BBCnJaaTJOrS',$,'Node 1',$,$,#16,#27,#23,$),
 #34=IfcStructuralPointConnection('08lPOE44r8VfWKpVuEhIr_',$,'Node 3',$,$,#16,#33,#29,$),
 #40=IfcStructuralPointConnection('1fMF6F8Q15vAtwp3XlXYpQ',$,'Node 4',$,$,#16,#39,#35,$),
 #45=IfcStructuralCurveMember('0GK40u5TD5nuzsl7$QTb_H',$,'Truss member',$,$,#16,#43,.PIN_JOINED_MEMBER.,#44),
 #52=IfcStructuralCurveMember('28D$6D$AD8_vHHIAz18ej6',$,'Truss member',$,$,#16,#50,.PIN_JOINED_MEMBER.,#51),
 #59=IfcStructuralCurveMember('1w1gl_0E13hBklt59T6yn0',$,'Truss member',$,$,#16,#57,.PIN_JOINED_MEMBER.,#58))

In [ ]:
nodes = [
    (node.GlobalId, node, node.AppliedCondition)
    for node in nodes_and_bars
    if node.is_a("IfcStructuralPointConnection")
]

In [ ]:
nodes[0][1].Representation.Representations[0].Items[
    0
].VertexGeometry.Coordinates

(0.0, 0.0, 3000.0)

In [ ]:
type(
    nodes[0][1]
    .Representation.Representations[0]
    .Items[0]
    .VertexGeometry.Coordinates
)

tuple

In [ ]:
np.array(
    nodes[0][1]
    .Representation.Representations[0]
    .Items[0]
    .VertexGeometry.Coordinates
)

array([   0.,    0., 3000.])

In [ ]:
pd.DataFrame(
    nodes, columns=["Node_GlobalId", "Ifc_Nodee", "Node_AppliedCondition"]
)

,Node_GlobalId,Ifc_Nodee,Node_AppliedCondition
0,2pRj5mp2n9meHLovYC9E37,"[2pRj5mp2n9meHLovYC9E37, None, Node 2, None, N...","[Support, [False], [True], [False], [False], [..."
1,0l8conwJL1BBCnJaaTJOrS,"[0l8conwJL1BBCnJaaTJOrS, None, Node 1, None, N...","[Support, [True], [True], [True], [False], [Fa..."
2,08lPOE44r8VfWKpVuEhIr_,"[08lPOE44r8VfWKpVuEhIr_, None, Node 3, None, N...","[Support, [True], [True], [True], [False], [Fa..."
3,1fMF6F8Q15vAtwp3XlXYpQ,"[1fMF6F8Q15vAtwp3XlXYpQ, None, Node 4, None, N...","[Support, [True], [True], [True], [False], [Fa..."


In [ ]:
bars = [
    (bar.GlobalId, bar)
    for bar in nodes_and_bars
    if bar.is_a("IfcStructuralCurveMember")
]

In [ ]:
bars[1][1].ConnectedBy

(#53=IfcRelConnectsStructuralMember('06Oc0UdYP3aBPF8mJbjTvs',$,'Joint',$,#52,#22,#12,$,$,$),
 #54=IfcRelConnectsStructuralMember('2EVElF$QbA9wtrYNw7WD7r',$,'Joint',$,#52,#34,#12,$,$,$))

In [ ]:
bars[1][1].Representation.Representations[0].Items[
    0
].EdgeStart.VertexGeometry.Coordinates

(0.0, 0.0, 3000.0)

In [ ]:
bars[1][1].HasAssociations[0].RelatingMaterial.MaterialProfiles

(#74=IfcMaterialProfile($,$,#68,#71,$,$),)

In [ ]:
# | export
@patch
def get_coordinates(self: View, node):
    coordinates = (
        node.Representation.Representations[0]
        .Items[0]
        .VertexGeometry.Coordinates
    )
    return coordinates

In [ ]:
# | export
@patch
def get_translational_x(self: View, node):
    translational_x = node.AppliedCondition.TranslationalStiffnessX[0]
    return translational_x

In [ ]:
# | export
@patch
def get_translational_y(self: View, node):
    translational_y = node.AppliedCondition.TranslationalStiffnessY[0]
    return translational_y

In [ ]:
# | export
@patch
def get_translational_z(self: View, node):
    translational_z = node.AppliedCondition.TranslationalStiffnessZ[0]
    return translational_z

In [ ]:
# | export
@patch
def get_start_node(self: View, bar):
    # This implementation cannot guarantee that it is really the start_node
    # defined by EdgeStart
    start_node = bar.ConnectedBy[0].RelatedStructuralConnection.GlobalId
    return start_node

In [ ]:
# | export
@patch
def get_end_node(self: View, bar):
    # This implementation cannot guarantee that it is really the end_node
    # defined by EdgeEnd
    end_node = bar.ConnectedBy[1].RelatedStructuralConnection.GlobalId
    return end_node

In [ ]:
(
    bars[1][1]
    .HasAssociations[0]
    .RelatingMaterial.MaterialProfiles[0]
    .Profile.HasProperties[0]
    .Properties[0]
    .NominalValue[0]
)

1000.0

In [ ]:
(
    bars[1][1]
    .HasAssociations[0]
    .RelatingMaterial.MaterialProfiles[0]
    .Profile.HasProperties[0]
    .Properties[0]
    .Unit
)

In [ ]:
# | export
@patch
def get_surface_area(self: View, bar):
    surface_area = (
        bar.HasAssociations[0]
        .RelatingMaterial.MaterialProfiles[0]
        .Profile.HasProperties[0]
        .Properties[0]
        .NominalValue[0]
    )
    return surface_area

In [ ]:
(
    bars[1][1]
    .HasAssociations[0]
    .RelatingMaterial.MaterialProfiles[0]
    .Material.HasProperties[0]
    .Properties[0]
    .NominalValue[0]
)

1000.0

In [ ]:
(
    bars[1][1]
    .HasAssociations[0]
    .RelatingMaterial.MaterialProfiles[0]
    .Material.HasProperties[0]
    .Properties[0]
)

#69=IfcPropertySingleValue('YoungModulus',$,IfcModulusOfElasticityMeasure(1000.),$)

In [ ]:
# | export
@patch
def get_modulus_of_elasticity(self: View, bar):
    modulus_of_elasticity = (
        bar.HasAssociations[0]
        .RelatingMaterial.MaterialProfiles[0]
        .Material.HasProperties[0]
        .Properties[0]
        .NominalValue[0]
    )
    return modulus_of_elasticity

In [ ]:
# | export
@patch
def get_nodes_and_bars(self: View):
    self.nodes_and_bars = self.ifc_structural_analysis_model.IsGroupedBy[
        0
    ].RelatedObjects
    self.nodes = [
        (
            node.GlobalId,
            *self.get_coordinates(node),
            self.get_translational_x(node),
            self.get_translational_y(node),
            self.get_translational_z(node),
        )
        for node in self.nodes_and_bars
        if node.is_a("IfcStructuralPointConnection")
    ]
    self.bars = [
        (
            bar.GlobalId,
            self.get_start_node(bar),
            self.get_end_node(bar),
            self.get_surface_area(bar),
            self.get_modulus_of_elasticity(bar),
        )
        for bar in self.nodes_and_bars
        if bar.is_a("IfcStructuralCurveMember")
    ]
    self.nodes_df = pd.DataFrame(
        self.nodes,
        columns=[
            "Node",
            "Coordinate_X",
            "Coordinate_Y",
            "Coordinate_Z",
            "Translational_X",
            "Translational_Y",
            "Translational_Z",
        ],
    )
    self.bars_df = pd.DataFrame(
        self.bars,
        columns=[
            "Bar",
            "Start_node",
            "End_node",
            "Cross-sectional_area",
            "Modulus_of_elasticity",
        ],
    )

In [ ]:
System.get_nodes_and_bars()

In [ ]:
System.nodes_df.style.hide(axis="index")

Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
2pRj5mp2n9meHLovYC9E37,0.000000,0.000000,3000.000000,False,True,False
0l8conwJL1BBCnJaaTJOrS,0.000000,0.000000,0.000000,True,True,True
08lPOE44r8VfWKpVuEhIr_,-4000.000000,0.000000,3000.000000,True,True,True
1fMF6F8Q15vAtwp3XlXYpQ,-4000.000000,0.000000,6000.000000,True,True,True


In [ ]:
System.bars_df.style.hide(axis="index")

Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
0GK40u5TD5nuzsl7$QTb_H,2pRj5mp2n9meHLovYC9E37,0l8conwJL1BBCnJaaTJOrS,1000.000000,1000.000000
28D$6D$AD8_vHHIAz18ej6,2pRj5mp2n9meHLovYC9E37,08lPOE44r8VfWKpVuEhIr_,1000.000000,1000.000000
1w1gl_0E13hBklt59T6yn0,2pRj5mp2n9meHLovYC9E37,1fMF6F8Q15vAtwp3XlXYpQ,1000.000000,1000.000000


In [ ]:
point_load = System.load_group.IsGroupedBy[0].RelatedObjects[0]

In [ ]:
point_load.AssignedToStructuralItem[0].RelatingElement

#22=IfcStructuralPointConnection('2pRj5mp2n9meHLovYC9E37',$,'Node 2',$,$,#16,#21,#17,$)

In [ ]:
point_load.AppliedLoad.ForceX

100000.0

In [ ]:
# | export
@patch
def get_force_x(self: View, point_load):
    force_x = point_load.AppliedLoad.ForceX
    return force_x

In [ ]:
# | export
@patch
def get_force_y(self: View, point_load):
    force_y = point_load.AppliedLoad.ForceY
    return force_y

In [ ]:
# | export
@patch
def get_force_z(self: View, point_load):
    force_z = point_load.AppliedLoad.ForceZ
    return force_z

In [ ]:
# | export
@patch
def get_node_for_point_load(self: View, point_load):
    node = point_load.AssignedToStructuralItem[0].RelatingElement.GlobalId
    return node

In [ ]:
# | export
@patch
def get_point_loads(self: View):
    self.point_loads = [
        (
            point_load.GlobalId,
            self.get_node_for_point_load(point_load),
            self.get_force_x(point_load),
            self.get_force_y(point_load),
            self.get_force_z(point_load),
        )
        for point_load in self.load_group.IsGroupedBy[0].RelatedObjects
    ]

    self.point_loads_df = pd.DataFrame(
        self.point_loads,
        columns=["Point_Load", "Node", "Force_X", "Force_Y", "Force_Z"],
    )

In [ ]:
System.get_point_loads()

In [ ]:
System.point_loads_df.style.hide(axis="index")

Point_Load,Node,Force_X,Force_Y,Force_Z
1EEwRxFIX0OwvpdEJv4Em2,2pRj5mp2n9meHLovYC9E37,100000.000000,0.000000,-100000.000000


# Result group

In [ ]:
ifctruss.ifctruss.solve(model)

In [ ]:
# | export
@patch
def get_result_group(self: View, result_group):
    self.result_group = self.ifc_model.by_guid(result_group)
    self.theory_type = self.result_group.TheoryType
    self.is_linear = self.result_group.IsLinear

In [ ]:
result_group = model.by_type("IfcStructuralResultGroup")[0].GlobalId

In [ ]:
System.get_result_group(result_group)

In [ ]:
System.theory_type

'FIRST_ORDER_THEORY'

In [ ]:
System.is_linear

True

In [ ]:
System.result_group.IsGroupedBy[0].RelatedObjects

(#79=IfcStructuralPointReaction('0mNpOoNuP8Wg3h25BnSTt6',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.),
 #82=IfcStructuralPointReaction('2$VUW7hXz9rBtzvUG2JSh9',$,'Result_Force',$,$,$,$,#81,.GLOBAL_COORDS.),
 #85=IfcStructuralPointReaction('3oWPZNTuf8FOoRE6N3QIra',$,'Result_Force',$,$,$,$,#84,.GLOBAL_COORDS.),
 #88=IfcStructuralPointReaction('3$8SvgKg9Bt9zwSOIFLHJz',$,'Result_Force',$,$,$,$,#87,.GLOBAL_COORDS.),
 #93=IfcStructuralCurveReaction('2kErAON3rDGQkfdMIwAk14',$,'Result_Normal_Force',$,$,$,$,#92,.LOCAL_COORDS.,.DISCRETE.),
 #98=IfcStructuralCurveReaction('1mIo5kJNn9gBGv0REQFP4b',$,'Result_Normal_Force',$,$,$,$,#97,.LOCAL_COORDS.,.DISCRETE.),
 #103=IfcStructuralCurveReaction('0b77Hjq5vEGf2QB8e$8oBv',$,'Result_Normal_Force',$,$,$,$,#102,.LOCAL_COORDS.,.DISCRETE.))

In [ ]:
System.result_group.IsGroupedBy[0].RelatedObjects[0].AppliedLoad

#78=IfcStructuralLoadSingleDisplacement('Result_Displacement',214.814814814815,0.,-195.833333333333,$,$,$)

In [ ]:
System.result_group.IsGroupedBy[0].RelatedObjects[
    0
].AssignedToStructuralItem[0].RelatingElement

#22=IfcStructuralPointConnection('2pRj5mp2n9meHLovYC9E37',$,'Node 2',$,$,#16,#21,#17,$)

In [ ]:
# | export
@patch
def get_displacement_x(self: View, point_reaction):
    displacement_x = point_reaction.AppliedLoad.DisplacementX
    return displacement_x

In [ ]:
# | export
@patch
def get_displacement_y(self: View, point_reaction):
    displacement_y = point_reaction.AppliedLoad.DisplacementY
    return displacement_y

In [ ]:
# | export
@patch
def get_displacement_z(self: View, point_reaction):
    displacement_z = point_reaction.AppliedLoad.DisplacementZ
    return displacement_z

In [ ]:
# | export
@patch
def get_node_for_displacement(self: View, point_reaction):
    node = point_reaction.AssignedToStructuralItem[
        0
    ].RelatingElement.GlobalId
    return node

In [ ]:
# | export
@patch
def get_displacements(self: View):
    self.point_reactions = [
        point_reaction
        for point_reaction in self.result_group.IsGroupedBy[
            0
        ].RelatedObjects
        if point_reaction.is_a("IfcStructuralPointReaction")
    ]

    self.displacments = [
        (
            self.get_node_for_displacement(point_reaction),
            self.get_displacement_x(point_reaction),
            self.get_displacement_y(point_reaction),
            self.get_displacement_z(point_reaction),
        )
        for point_reaction in self.point_reactions
        if point_reaction.AppliedLoad.is_a(
            "IfcStructuralLoadSingleDisplacement"
        )
    ]

    self.displacments_df = pd.DataFrame(
        self.displacments,
        columns=[
            "Node",
            "Displacement_X",
            "Displacement_Y",
            "Displacement_Z",
        ],
    )

In [ ]:
System.get_displacements()

In [ ]:
System.point_reactions

[#79=IfcStructuralPointReaction('0mNpOoNuP8Wg3h25BnSTt6',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.),
 #82=IfcStructuralPointReaction('2$VUW7hXz9rBtzvUG2JSh9',$,'Result_Force',$,$,$,$,#81,.GLOBAL_COORDS.),
 #85=IfcStructuralPointReaction('3oWPZNTuf8FOoRE6N3QIra',$,'Result_Force',$,$,$,$,#84,.GLOBAL_COORDS.),
 #88=IfcStructuralPointReaction('3$8SvgKg9Bt9zwSOIFLHJz',$,'Result_Force',$,$,$,$,#87,.GLOBAL_COORDS.)]

In [ ]:
System.displacments

[('2pRj5mp2n9meHLovYC9E37', 214.8148148148148, 0.0, -195.83333333333334)]

In [ ]:
System.displacments_df

,Node,Displacement_X,Displacement_Y,Displacement_Z
0,2pRj5mp2n9meHLovYC9E37,214.814815,0.0,-195.833333


In [ ]:
# | export
@patch
def get_forces(self: View):
    self.forces = [
        (
            # Since they are similar relationships,
            # the methods used for Point_load are reused.
            self.get_node_for_point_load(point_reaction),
            self.get_force_x(point_reaction),
            self.get_force_y(point_reaction),
            self.get_force_z(point_reaction),
        )
        for point_reaction in self.point_reactions
        if point_reaction.AppliedLoad.is_a("IfcStructuralLoadSingleForce")
    ]

    self.forces_df = pd.DataFrame(
        self.forces,
        columns=[
            "Node",
            "Force_X",
            "Force_Y",
            "Force_Z",
        ],
    )

In [ ]:
System.get_forces()

In [ ]:
System.forces_df

,Node,Force_X,Force_Y,Force_Z
0,0l8conwJL1BBCnJaaTJOrS,0.000000,0.0,65277.777778
1,08lPOE44r8VfWKpVuEhIr_,-53703.703704,0.0,0.000000
2,1fMF6F8Q15vAtwp3XlXYpQ,-46296.296296,0.0,34722.222222


In [ ]:
# | export
@patch
def get_normal_force(self: View, curve_reaction):
    first_location = curve_reaction.AppliedLoad.Locations[0][0]
    second_location = curve_reaction.AppliedLoad.Locations[1][0]

    first_value = curve_reaction.AppliedLoad.Values[0]
    second_value = curve_reaction.AppliedLoad.Values[1]

    if first_location < second_location:
        normal_force = second_value.ForceX
    elif first_location > second_location:
        normal_force = first_value.ForceX

    if normal_force > 0:
        type_of_normal_force = "Tensile force"
    elif normal_force < 0:
        type_of_normal_force = "Compressive force"
    elif normal_force == 0:
        type_of_normal_force = "Zero-force"

    return normal_force, type_of_normal_force

In [ ]:
# | export
@patch
def get_normal_forces(self: View):
    self.curve_reactions = [
        curve_reaction
        for curve_reaction in self.result_group.IsGroupedBy[
            0
        ].RelatedObjects
        if curve_reaction.is_a("IfcStructuralCurveReaction")
    ]

    self.normal_forces = [
        (
            # Since they are similar relationships,
            # the methods used for Point_load are reused.
            self.get_node_for_point_load(curve_reaction),
            *self.get_normal_force(curve_reaction),
        )
        for curve_reaction in self.curve_reactions
    ]

    self.normal_forces_df = pd.DataFrame(
        self.normal_forces,
        columns=[
            "Bar",
            "Normal_force",
            "Type_of_normal_force",
        ],
    )

In [ ]:
System.get_normal_forces()

In [ ]:
System.normal_forces_df

,Bar,Normal_force,Type_of_normal_force
0,0GK40u5TD5nuzsl7$QTb_H,-65277.777778,Compressive force
1,28D$6D$AD8_vHHIAz18ej6,53703.703704,Tensile force
2,1w1gl_0E13hBklt59T6yn0,57870.370370,Tensile force


In [ ]:
System.curve_reactions

[#93=IfcStructuralCurveReaction('2kErAON3rDGQkfdMIwAk14',$,'Result_Normal_Force',$,$,$,$,#92,.LOCAL_COORDS.,.DISCRETE.),
 #98=IfcStructuralCurveReaction('1mIo5kJNn9gBGv0REQFP4b',$,'Result_Normal_Force',$,$,$,$,#97,.LOCAL_COORDS.,.DISCRETE.),
 #103=IfcStructuralCurveReaction('0b77Hjq5vEGf2QB8e$8oBv',$,'Result_Normal_Force',$,$,$,$,#102,.LOCAL_COORDS.,.DISCRETE.)]

In [ ]:
test_value = System.get_node_for_point_load(System.curve_reactions[0])

In [ ]:
(test_value, *System.get_normal_force(System.curve_reactions[0]))

('0GK40u5TD5nuzsl7$QTb_H', -65277.777777777774, 'Compressive force')

In [ ]:
System.curve_reactions[0].AppliedLoad.Locations[1][0]

3000.0

In [ ]:
System.curve_reactions[0].AppliedLoad.Values[1].ForceX

-65277.777777777774

In [ ]:
# | hide
nbdev.nbdev_export()